In [1]:
# Import required libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

# from google.colab import files
# uploaded = files.upload()
from google.colab import drive
drive.mount('/content/drive')

# Load the dataset
data = pd.read_csv('/content/drive/My Drive/[[[[ECCE 2025 CUET]]]]/Implementation of Existing Model/online_shoppers_intention.csv')

# Define numerical and categorical columns
numerical_columns = ['Administrative', 'Administrative_Duration', 'Informational', 'Informational_Duration',
                     'ProductRelated', 'ProductRelated_Duration', 'BounceRates', 'ExitRates', 'PageValues', 'SpecialDay']
categorical_columns = ['Month', 'OperatingSystems', 'Browser', 'Region', 'TrafficType', 'VisitorType', 'Weekend']

# Separate features and target variable
X = data[numerical_columns + categorical_columns]
y = data['Revenue']  # Assuming 'Revenue' is the dependent variable

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Mounted at /content/drive


**Preprocess the data**

In [2]:
# Preprocess the data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_columns),
        ('cat', OneHotEncoder(drop='first'), categorical_columns)
    ]
)

X_train_transformed = preprocessor.fit_transform(X_train)
X_test_transformed = preprocessor.transform(X_test)

**Train the Linear Regression model and make prediction**

In [4]:
# Step 4: Train the Random Forest classifier
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train_transformed, y_train) # Changed X_train to X_train_transformed

# Step 5: Evaluate the model
y_pred = rf_model.predict(X_test_transformed) # Changed X_test to X_test_transformed

**Evaluate the model**

In [5]:
# Calculate accuracy and print evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")
print("Classification Report:")
print(classification_report(y_test, y_pred))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

Accuracy: 0.89
Classification Report:
              precision    recall  f1-score   support

       False       0.91      0.97      0.94      2055
        True       0.78      0.51      0.62       411

    accuracy                           0.89      2466
   macro avg       0.85      0.74      0.78      2466
weighted avg       0.89      0.89      0.89      2466

Confusion Matrix:
[[1996   59]
 [ 200  211]]


**Feature Importance (Coefficients)**

In [7]:
# Feature Importance (Coefficients)
feature_names = (
    numerical_columns +
    preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_columns).tolist()
)


# Decision Trees don't have coefficients like linear models.
# We'll use feature_importances_ instead.
feature_importances = rf_model.feature_importances_  # Get feature importances from the trained Decision Tree

coefficients = pd.DataFrame({
    'Feature': feature_names,
    'Importance': feature_importances  # Use 'Importance' instead of 'Coefficient'
}).sort_values(by='Importance', ascending=False)  # Sort by importance

print("\nFeature Importance:")
print(coefficients)


Feature Importance:
                    Feature    Importance
8                PageValues  3.327216e-01
5   ProductRelated_Duration  8.526129e-02
7                 ExitRates  8.471626e-02
4            ProductRelated  7.253378e-02
6               BounceRates  5.450867e-02
..                      ...           ...
35               Browser_11  3.005225e-05
62           TrafficType_18  6.579420e-07
33                Browser_9  0.000000e+00
56           TrafficType_12  0.000000e+00
61           TrafficType_17  0.000000e+00

[68 rows x 2 columns]


In [8]:
# Make predictions and convert to binary using a threshold (e.g., 0.5)
y_pred = rf_model.predict(X_test_transformed) # Changed 'model' to 'dt_model'
y_pred_binary = np.where(y_pred > 0.5, 1, 0)  # Convert to 0 or 1 based on threshold

# Calculate classification metrics using the binary predictions
accuracy = accuracy_score(y_test, y_pred_binary)
precision = precision_score(y_test, y_pred_binary)
recall = recall_score(y_test, y_pred_binary)
f1 = f1_score(y_test, y_pred_binary)

print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

Accuracy: 0.89
Precision: 0.78
Recall: 0.51
F1 Score: 0.62
